# ToDo -> Мета-моделирования (Meta-Labeling)


In [1]:
import pandas as pd
import numpy as np
import json
import os
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
# from lightgbm import LGBMClassifier

import joblib

from datetime import datetime, timezone
from sklearn.metrics import roc_auc_score


from meta_model_trainer import (
    find_latest_model_artifacts,
    get_out_of_sample_predictions,
    train_meta_model,
    validate_meta_model,
    evaluate_final_strategy,
    META_CONFIDENCE_THRESHOLD,
    # save_meta_model
)


In [2]:
# META_FEATURES = [
#     'pred_proba_buy', 'pred_proba_sell', 'proba_diff', 'atr_14', 'rsi_14'
# ]

In [3]:
# # ==============================================================================
# # --- ИЗМЕНЕНИЕ: ГЛОБАЛЬНЫЙ ПАРАМЕТР ДЛЯ МЕТА-МОДЕЛИ ---
# # Порог уверенности для мета-модели. Она "одобрит" сделку, только если
# # ее уверенность в том, что первичный сигнал верный, будет выше этого порога.
# META_CONFIDENCE_THRESHOLD = 0.55
# # ==============================================================================

# # ... (функция find_latest_model_artifacts остается без изменений) ...
# def find_latest_model_artifacts(folder_path="../models/"):
#     metadata_files = [f for f in os.listdir(folder_path) if f.endswith("_metadata.json")]
#     if not metadata_files:
#         raise FileNotFoundError("В папке models не найдено ни одного файла метаданных (*_metadata.json).")
#     latest_metadata_file = max(metadata_files)
#     base_path = os.path.join(folder_path, latest_metadata_file.replace("_metadata.json", ""))
#     return base_path


In [4]:


# def get_out_of_sample_predictions(X, y, primary_model, n_splits=5):
#     print(f"Начинаем TimeSeriesSplit Cross-Validation с {n_splits} сплитами...")
#     cv_splitter = TimeSeriesSplit(n_splits=n_splits)
#     oof_preds = pd.DataFrame(index=X.index, columns=['pred_proba_sell', 'pred_proba_hold', 'pred_proba_buy', 'primary_pred'])

#     for i, (train_idx, test_idx) in enumerate(cv_splitter.split(X)):
#         print(f"--- Сплит {i+1}/{n_splits} ---")
#         X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
#         y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
#         primary_model.fit(X_train, y_train)
        
#         probas = primary_model.predict_proba(X_test)
#         preds = primary_model.predict(X_test)
        
#         oof_preds.iloc[test_idx, 0:3] = probas
#         oof_preds.iloc[test_idx, 3] = preds

#     oof_preds.dropna(inplace=True)
#     print("Генерация 'честных' предсказаний завершена.")
#     return oof_preds



In [5]:
# # ==============================================================================
# # НОВАЯ ФУНКЦИЯ: ВАЛИДАЦИЯ МЕТА-МОДЕЛИ
# # ==============================================================================

# def validate_meta_model(X_meta, y_meta):
#     """
#     Проводит Walk-Forward валидацию для мета-модели, чтобы оценить ее реальный навык.
#     """
#     print("\n--- Проведение Walk-Forward Валидации для МЕТА-МОДЕЛИ ---")
    
#     n_splits = 5
#     tscv = TimeSeriesSplit(n_splits=n_splits)
#     meta_scores_auc = []
#     meta_scores_f1 = []

#     for i, (train_index, test_index) in enumerate(tscv.split(X_meta)):
#         X_meta_train, X_meta_test = X_meta.iloc[train_index], X_meta.iloc[test_index]
#         y_meta_train, y_meta_test = y_meta.iloc[train_index], y_meta.iloc[test_index]
        
#         # Проверяем, есть ли в тестовой выборке оба класса (0 и 1)
#         if len(y_meta_test.unique()) < 2:
#             print(f"--- Мета-сплит {i+1}/{n_splits} --- Пропущен (только один класс в тестовой выборке).")
#             continue

#         meta_model_cv = LogisticRegression(class_weight='balanced', random_state=42)
#         meta_model_cv.fit(X_meta_train, y_meta_train)
        
#         # Предсказываем вероятности для AUC
#         y_meta_pred_proba = meta_model_cv.predict_proba(X_meta_test)[:, 1]
        
#         print(f"\n--- Мета-сплит {i+1}/{n_splits} ---")
#         # ROC AUC - главная метрика для оценки бинарных классификаторов
#         auc_score = roc_auc_score(y_meta_test, y_meta_pred_proba)
#         meta_scores_auc.append(auc_score)
#         print(f"ROC AUC: {auc_score:.4f}")
        
#         # Также посмотрим на classification_report для детального анализа
#         y_meta_pred = meta_model_cv.predict(X_meta_test)
#         report = classification_report(y_meta_test, y_meta_pred, target_names=['0 (Bad Signal)', '1 (Good Signal)'], output_dict=True)
#         meta_scores_f1.append(report['1 (Good Signal)']['f1-score'])
#         print(classification_report(y_meta_test, y_meta_pred, target_names=['0 (Bad Signal)', '1 (Good Signal)']))

#     print("-" * 60)
#     print(f"Средний ROC AUC мета-модели по всем сплитам: {np.mean(meta_scores_auc):.4f}")
#     if np.mean(meta_scores_auc) > 0.5:
#         print("-> Результат > 0.5, мета-модель обладает предсказательной силой.")
#     else:
#         print("-> Результат <= 0.5, мета-модель не лучше случайного угадывания.")
#     print(f"Средний F1-score для класса 'Good Signal': {np.mean(meta_scores_f1):.4f}")
#     print("-" * 60)

In [6]:
# # ==============================================================================
# # ШАГ 2: СОЗДАНИЕ МЕТА-МЕТОК И ОБУЧЕНИЕ МЕТА-МОДЕЛИ (УЛУЧШЕННАЯ ВЕРСИЯ)
# # ==============================================================================

# # --- ИЗМЕНЕНИЕ: Функция теперь принимает полный DataFrame `df` ---
# def train_meta_model(oof_preds, df):
#     """Создает мета-метки и обучает мета-модель на обогащенном наборе признаков."""
#     print("\n--- Обучение мета-модели ---")
    
#     # Объединяем "честные" предсказания с полным датафреймом, чтобы получить доступ к исходным признакам
#     full_df = oof_preds.join(df)
    
#     # 1. Отбираем все торговые сигналы (+1 или -1)
#     df_filtered = full_df[full_df['primary_pred'] != 0].copy()
        
#     if df_filtered.empty:
#         print("ВНИМАНИЕ: Первичная модель не сгенерировала ни одного торгового сигнала.")
#         return None

#     # 2. Создаем мета-метки (y_meta): 1, если угадали, 0 - если ошиблись
#     df_filtered['y_meta'] = (df_filtered['primary_pred'] == df_filtered[TARGET_COLUMN]).astype(int)
    
#     print(f"Всего сигналов для обучения мета-модели: {len(df_filtered)}")
#     print("Распределение мета-меток (1=верно, 0=неверно):")
#     print(df_filtered['y_meta'].value_counts(normalize=True).round(2))
    
#     # --- ИЗМЕНЕНИЕ: Создаем обогащенный набор признаков для мета-модели ---
#     # 3. Определяем признаки для мета-модели (X_meta)
#     df_filtered['proba_diff'] = df_filtered['pred_proba_buy'] - df_filtered['pred_proba_sell']
    
    
#     # Проверяем, что все нужные признаки есть
#     if not all(feat in df_filtered.columns for feat in META_FEATURES):
#         raise ValueError("Не все признаки для мета-модели найдены в DataFrame.")

#     X_meta = df_filtered[META_FEATURES]
#     y_meta = df_filtered['y_meta']
#     print(f"Мета-модель будет обучаться на {len(META_FEATURES)} признаках.")

#     # 4. Обучаем простую мета-модель
#     meta_model = LogisticRegression(class_weight='balanced')
#     print("Обучение LogisticRegression...")
#     meta_model.fit(X_meta, y_meta)
    
#     print("Обучение мета-модели завершено.")
#     return meta_model



In [7]:
# # ==============================================================================
# # ШАГ 3: ОЦЕНКА ФИНАЛЬНОЙ СТРАТЕГИИ (УЛУЧШЕННАЯ ВЕРСИЯ)
# # ==============================================================================

# # --- ИЗМЕНЕНИЕ: Функция теперь принимает полный DataFrame `df` ---
# def evaluate_final_strategy(oof_preds, df, meta_model):
#     """Оценивает итоговую стратегию, отфильтрованную мета-моделью с порогом уверенности."""
#     print("\n--- Оценка финальной стратегии (Первичная модель + Мета-модель) ---")
    
#     full_df = oof_preds.join(df)
    
#     # 1. Отбираем все торговые сигналы от первичной модели
#     signals_to_filter = full_df[full_df['primary_pred'] != 0].copy()
#     if signals_to_filter.empty:
#         print("Нет торговых сигналов для оценки.")
#         return
        
#     # --- ИЗМЕНЕНИЕ: Создаем те же обогащенные признаки для предсказания ---
#     signals_to_filter['proba_diff'] = signals_to_filter['pred_proba_buy'] - signals_to_filter['pred_proba_sell']
    
#     if not all(feat in signals_to_filter.columns for feat in META_FEATURES):
#         raise ValueError("Не все признаки для предсказания мета-моделью найдены.")

#     X_meta_live = signals_to_filter[META_FEATURES]

#     # --- ИЗМЕНЕНИЕ: Применяем порог уверенности ---
#     # 2. Получаем вероятности от мета-модели
#     meta_probas = meta_model.predict_proba(X_meta_live)
#     # Вероятность того, что сигнал верный (класс 1)
#     proba_signal_is_correct = meta_probas[:, 1]
    
#     # 3. Отбираем сделки, где мета-модель достаточно уверена
#     approved_mask = proba_signal_is_correct > META_CONFIDENCE_THRESHOLD
#     final_trades = signals_to_filter[approved_mask]
    
#     print(f"Всего первичных торговых сигналов: {len(signals_to_filter)}")
#     print(f"Сигналов, одобренных мета-моделью (уверенность > {META_CONFIDENCE_THRESHOLD:.0%}): {len(final_trades)}")
    
#     if len(final_trades) == 0:
#         print("Мета-модель не одобрила ни одного сигнала. Попробуйте снизить META_CONFIDENCE_THRESHOLD.")
#         return
        
#     # 4. Считаем итоговые метрики
#     final_y_true = final_trades[TARGET_COLUMN]
#     final_y_pred = final_trades['primary_pred']
    
#     y_true_int = final_y_true.astype(int)
#     y_pred_int = final_y_pred.astype(int)
#     all_possible_labels = [-1, 0, 1]
    
#     print("\nОтчет по качеству для финальных, отфильтрованных сделок:")
#     print(classification_report(
#         y_true_int, y_pred_int, labels=all_possible_labels, 
#         target_names=['-1 (Sell)', '0 (Hold)', '1 (Buy)'], zero_division=0
#     ))
    
#     win_rate = accuracy_score(y_true_int, y_pred_int)
#     print("--------------------------------------------------------------------------")
#     print(f"Ключевая метрика: Общий Win Rate (Точность) одобренных сделок: {win_rate:.2%}")

#     return win_rate



In [ ]:
import pandas as pd, json, os

# === Загрузка данных ===
df = pd.read_csv("../data/moex_final_dataset.csv")
df.sort_values("Date", inplace=True)
df.reset_index(drop=True, inplace=True)

# === Загрузка базовой модели ===
base_path = find_latest_model_artifacts()
metadata = json.load(open(f"{base_path}_metadata.json", "r", encoding="utf-8"))
feature_cols = json.load(open(f"{base_path}_features.json", "r"))
TARGET_COLUMN = metadata["target_column"]

In [9]:
df[TARGET_COLUMN].fillna(value=0, inplace=True)

/var/folders/lh/9k36l4vn35v4qnj9z669pmtc0000gn/T/ipykernel_24018/3218857855.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[TARGET_COLUMN].fillna(value=0, inplace=True)


In [10]:
from LGBMClassifier_modeling import build_model
X = df[feature_cols]
y = df[TARGET_COLUMN].astype(int)
# primary_model = LGBMClassifier(**metadata["model_parameters"])
primary_model = build_model()

# === 1. Получаем "честные" предсказания ===
oof = get_out_of_sample_predictions(X, y, primary_model)

# === 2. Обучаем мета-модель ===
meta_model = train_meta_model(oof, df, TARGET_COLUMN)

# === 3. Валидируем ===
df_filtered = oof.join(df)
df_filtered['proba_diff'] = df_filtered['pred_proba_buy'] - df_filtered['pred_proba_sell']
X_meta = df_filtered[['pred_proba_buy', 'pred_proba_sell', 'proba_diff', 'atr_14', 'rsi_14']]
y_meta = (df_filtered['primary_pred'] == df_filtered[TARGET_COLUMN]).astype(int)

validate_meta_model(X_meta, y_meta)

# === 4. Оцениваем финальную стратегию ===
evaluate_final_strategy(oof, df, meta_model, TARGET_COLUMN)



Начинаем TimeSeriesSplit с 5 сплитами...
--- Сплит 1/5 ---
--- Сплит 2/5 ---
--- Сплит 3/5 ---
--- Сплит 4/5 ---
--- Сплит 5/5 ---
Сплит 1: AUC=0.7111
Сплит 2: AUC=0.6790
Сплит 3: AUC=0.6759
Сплит 4: AUC=0.6998
Сплит 5: AUC=0.6294
Средний AUC=0.6790, F1=0.5816
✅ Всего сигналов: 21513, одобрено мета-моделью: 21508
              precision    recall  f1-score   support

        Sell       0.42      0.46      0.44      8817
        Hold       0.00      0.00      0.00      2924
         Buy       0.44      0.54      0.49      9767

    accuracy                           0.43     21508
   macro avg       0.29      0.33      0.31     21508
weighted avg       0.37      0.43      0.40     21508

Win Rate: 43.14%


/Users/beilakaliev/Library/Caches/pypoetry/virtualenvs/ml-modeling-TB8BmMSm-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: divide by zero encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/beilakaliev/Library/Caches/pypoetry/virtualenvs/ml-modeling-TB8BmMSm-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: overflow encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/beilakaliev/Library/Caches/pypoetry/virtualenvs/ml-modeling-TB8BmMSm-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: invalid value encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/beilakaliev/Library/Caches/pypoetry/virtualenvs/ml-modeling-TB8BmMSm-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: divide by zero encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/

0.43142086665426815

In [11]:
# --- ШАГ 4: Сохранение артефактов мета-модели ---
if meta_model:
    print("\n--- Сохранение артефактов мета-модели ---")
    
    # Определяем папку и имя файла, связывая его с первичной моделью
    output_folder = "../models/meta"
    # primary_model_name = os.path.basename(latest_model_base_path)
    filename_base = "final_model"

    # 1. Сохранение самой мета-модели
    meta_model_path = os.path.join(output_folder, f"{filename_base}.joblib")
    joblib.dump(meta_model, meta_model_path)
    print(f"Мета-модель сохранена в: {meta_model_path}")

    # 2. Сохранение метаданных мета-модели
    meta_metadata_path = os.path.join(output_folder, f"{filename_base}_metadata.json")
    
    # Собираем важную информацию
    meta_metadata = {
        "meta_model_name": "trade_signal_filter",
        "meta_model_class": type(meta_model).__name__,
        # "primary_model_used": primary_model_name,
        "training_timestamp_utc": datetime.now(timezone.utc).isoformat(),
        "meta_features_used": ['pred_proba_buy', 'pred_proba_sell', 'proba_diff', 'atr_14', 'rsi_14'],
        "meta_confidence_threshold": META_CONFIDENCE_THRESHOLD,
        "model_parameters": meta_model.get_params(),
        "target_column": TARGET_COLUMN,
    }

    with open(meta_metadata_path, 'w', encoding='utf-8') as f:
        json.dump(meta_metadata, f, indent=4, ensure_ascii=False)
    print(f"Метаданные мета-модели сохранены в: {meta_metadata_path}")


    features_path = os.path.join(output_folder, f"{filename_base}_features.json")
    features_list = list(X.columns)
    with open(features_path, 'w') as f:
        json.dump(features_list, f, indent=4)
    print(f"Список признаков сохранен в: {features_path}")



--- Сохранение артефактов мета-модели ---
Мета-модель сохранена в: ../models/meta/final_model.joblib
Метаданные мета-модели сохранены в: ../models/meta/final_model_metadata.json
Список признаков сохранен в: ../models/meta/final_model_features.json
